In [ ]:
!git clone https://github.com/bpkwee/TCR_reconstruction
!pip install pysam

In [ ]:
# Full length tcr reconstruction
In this notebook an example is given on how to reconstruct full length tcr sequences from V, J and CDR3 annotations.

The reconstruction algorithm has been benchmarked on an internal dataset (RootPath), and an publically available dataset (10x).
Compared to the reconstruction of RootPath this reconstruction method matched near 100%$^1$ of their reconstructions (900+ TCRA and TCRB).
The 10x data was biological sample of 10k TCRS (50/50 TCRA/TCRB). The reconstruction fidelity of this dataset was $>85\%$
The remaining $15\%$ was explained by biological and/or technical noise and by missing info for allelic differences (alleles are often not annotated).

1. small differences were explained by a possible error and difference in assumptions between the unknown RootPath script and this method.

## Requirements
- a .csv file that contains the following columns (can be obtained by exporting/saving an excel file as .csv)
    - V and J annotations columns should contain the following column names: `TRAV', 'TRAJ',  'TRBV' and 'TRBJ`
    - CDR3a and CDR3b annotations should contain the following column names: `'cdr3_alpha_aa', 'cdr3_beta_aa'`
- two translation dictionaries
    - Multiple versions can be found in `TCR_reconstruction/data/VDJ_gene_sequences/IMGT_versions`
    - These translate (ambiguous) annotations to IMGT standardized format, and takes the corresponding V and J sequences.
    - When no
    - One should contain the IMGT annoations as keys with the $AA$ (amino acid) sequences as values
    - The other should contain the IMGT annoations as keys with the $NT$ (nucleotide) sequences as values


# Full length tcr reconstruction
In this notebook an example is given on how to reconstruct full length tcr sequences from V, J and CDR3 annotations.

The reconstruction algorithm has been benchmarked on an internal dataset (RootPath), and an publically available dataset (10x).
Compared to the reconstruction of RootPath this reconstruction method matched near 100%$^1$ of their reconstructions (900+ TCRA and TCRB).
The 10x data was biological sample of 10k TCRS (50/50 TCRA/TCRB). The reconstruction fidelity of this dataset was $>85\%$
The remaining $15\%$ was explained by biological and/or technical noise and by missing info for allelic differences (alleles are often not annotated).

1. small differences were explained by a possible error and difference in assumptions between the unknown RootPath script and this method.

## Requirements
- a .csv file that contains the following columns (can be obtained by exporting/saving an excel file as .csv)
    - V and J annotations columns should contain the following column names: `TRAV', 'TRAJ',  'TRBV' and 'TRBJ`
    - CDR3a and CDR3b annotations should contain the following column names: `'cdr3_alpha_aa', 'cdr3_beta_aa'`
- two translation dictionaries
    - Multiple versions can be found in `TCR_reconstruction/data/VDJ_gene_sequences/IMGT_versions`
    - These translate (ambiguous) annotations to IMGT standardized format, and takes the corresponding V and J sequences.
    - When no
    - One should contain the IMGT annoations as keys with the $AA$ (amino acid) sequences as values
    - The other should contain the IMGT annoations as keys with the $NT$ (nucleotide) sequences as values


In [2]:
# import the reconstruct_full_tcr function
from TCR_reconstruction.logger.logger import init_logger
from TCR_reconstruction.vdj_reconstruction_utils import reconstruct_full_tcr, reconstruct_vdj

logger = init_logger('TCR_reconstruction.log', level_msg='INFO')

Here the file paths should be given

Example:
```
saved_dataset_file_path = '../TCR_reconstruction/data/saved_data/2021-11-11_15h_final.csv'
translation_dict_aa = '../TCR_reconstruction/data/VDJ_gene_sequences/IMGT_versions/after_benchmark/functional_with_L-PART1+V-EXON_after_benchmark/vdj_translation_dict_aa_2021-12-05_18h_.json'
translation_dict_nt = '../TCR_reconstruction/data/VDJ_gene_sequences/IMGT_versions/after_benchmark/functional_with_L-PART1+V-EXON_after_benchmark/vdj_translation_dict_nt_2021-12-05_18h_.json'
```

In [4]:
saved_dataset_file_path = '../TCR_reconstruction/data/saved_data/2021-11-11_15h_final.csv'
translation_dict_aa = '../TCR_reconstruction/data/VDJ_gene_sequences/IMGT_versions/after_benchmark/functional_with_L-PART1+V-EXON_after_benchmark/vdj_translation_dict_aa_2021-12-05_18h_.json'
translation_dict_nt = '../TCR_reconstruction/data/VDJ_gene_sequences/IMGT_versions/after_benchmark/functional_with_L-PART1+V-EXON_after_benchmark/vdj_translation_dict_nt_2021-12-05_18h_.json'
dataset = pandas.read_csv(saved_dataset_file_path, low_memory=False)

Reconstruct the v,d and j aa and nt sequences from TRAV, TRAJ, TRBV, TRBJ annotations


In [5]:
for vdj in ['TRAV', 'TRAJ', 'TRBV', 'TRBD', 'TRBJ']:
    dataset[vdj + '_imgt_aa'],  dataset[vdj + '_seq_aa'],  dataset[
        vdj + '_imgt_nt'],  dataset[vdj + '_seq_nt'] = reconstruct_vdj(dataset,
                                                                       vdj,
                                                                       translation_dict_nt,
                                                                       translation_dict_aa)

Calculate how many annotations could be matched to a IMGT sequence:

In [6]:
total_len = len(  dataset)
for nt_or_aa in ['aa', 'nt']:
    for vdj in ['TRAV', 'TRAJ', 'TRBV', 'TRBD', 'TRBJ']:
        count = sum(  dataset[vdj + '_seq_' + nt_or_aa].notna())
        count_original = sum(  dataset[vdj].notna())
        print('{0} imputed: {1} / {4} total annotations ({2}) ({3})'.format(vdj, count, total_len,
                                                                                  nt_or_aa.upper(),
                                                                                  count_original))

TRAV imputed: 79138 / 79875 total annotations (129664) (AA)
TRAJ imputed: 79567 / 79748 total annotations (129664) (AA)
TRBV imputed: 115826 / 118060 total annotations (129664) (AA)
TRBD imputed: 55734 / 74483 total annotations (129664) (AA)
TRBJ imputed: 106635 / 106786 total annotations (129664) (AA)
TRAV imputed: 79157 / 79875 total annotations (129664) (NT)
TRAJ imputed: 78796 / 79748 total annotations (129664) (NT)
TRBV imputed: 115887 / 118060 total annotations (129664) (NT)
TRBD imputed: 55734 / 74483 total annotations (129664) (NT)
TRBJ imputed: 106635 / 106786 total annotations (129664) (NT)


reconstructing the full sequence for the beta and alpha TCR

In [9]:
# beta
dataset['full_seq_reconstruct_beta_aa'] = reconstruct_full_tcr(dataset['TRBV_seq_nt'],
                                                               dataset['TRBV_seq_aa'],
                                                               dataset['TRBJ_seq_nt'],
                                                               dataset['TRBJ_seq_aa'],
                                                               dataset['cdr3_beta_aa'],
                                                               include_leader=False)
# alpha
dataset['full_seq_reconstruct_alpha_aa'] = reconstruct_full_tcr(dataset['TRAV_seq_nt'],
                                                                dataset['TRAV_seq_aa'],
                                                                dataset['TRAJ_seq_nt'],
                                                                dataset['TRAJ_seq_aa'],
                                                                dataset['cdr3_alpha_aa'],
                                                                include_leader=False)

Calculate how many could be reconstructed:

In [10]:
print('Could reconstruct full BETA TCR for {0} entries of total {1} CDR3b entries'.format(
    sum(dataset['full_seq_reconstruct_beta_aa'].notna()),
    sum(   dataset['cdr3_beta_aa'].notna())))

print('Could  reconstruct full ALPHA TCR for {0} entries of total {1} CDR3a entries'.format(
    sum(   dataset['full_seq_reconstruct_alpha_aa'].notna()),
    sum(   dataset['cdr3_alpha_aa'].notna())))



Could reconstruct full BETA TCR for 104275 entries of total 119093 CDR3b entries
Could  reconstruct full ALPHA TCR for 76528 entries of total 80956 CDR3a entries
